# References

[1]: Deckers, Lucas, et al. "EEG-based detection of the attended speaker and the locus of auditory attention with convolutional neural networks." bioRxiv (2018): 475673.

# Library and File Imports

In [2]:
from google.colab import drive
drive.mount('gdrive', force_remount=True)

import sys
import os
sys.path.append(os.path.abspath('/content/gdrive/MyDrive/code/Phase_2_Part_2_CNN'))
# after changing any python file, Runtime -> Restart and run all
from dataGenerator import dataGenerator 
# from network import baseline_model
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import numpy as np
import pickle
from random import randint
import tensorflow as tf
import pandas as pd
import scipy.io as sio
import h5py
from scipy.io import loadmat
import keras
import keras.backend as K
from keras.initializers import RandomUniform
from keras.models import Sequential, load_model
from keras.utils import to_categorical
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Dense, Softmax, Activation, BatchNormalization, \
                         Conv2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.models import Model

Mounted at gdrive


# Parameters Definition

## File Related Parameters

In [3]:
number_of_subjects   = 16
tag_EEG              = 'output_Eeg'     # in case if switched in the future
tag_stimuli          = 'Stimuli_eeg'    # in case if switched in the future
tag_attended_track   = 'Attended_track' # in case if switched in the future
EEG_dir              = r'/content/gdrive/MyDrive/data/EEG_data_Nicolas/' 
EEG_test_dir         = None             # with no test set given
audio_dir            = r'/content/gdrive/MyDrive/data/Audio_data/'
model_save_dir       = r'/content/gdrive/MyDrive/results/Phase_2/CNN/model/'
experiment_folder    = r'experiment_1/' # modify if you change the experiment
experiment_save_dir  = model_save_dir + experiment_folder
checkpoint_save_dir  = experiment_save_dir + 'checkpoints/'

## Dataset Generation Related Parameters

In [4]:
window_time = 10 # denoted in seconds. 
# choices of window time: [10, 5, 2, 1] denoted in paper .

verbose = True # display information, True for more information. 

bool_validation = True # whether to split validation set from the dataset.

bool_test_set = False # 

'''whether to add the preprocessed audio envelope onto the network's input data.
if True, then the number of channels of one sample will be [window_length, 66], 
else, will be [window_length, 64]. 
'''
bool_envelope = True

'''the ratio between the overlapping part of the windows and the length of the 
window. If the ratio is higher, the more samples you will get. However, due to 
the limited RAM of Colaboratory, the maximum value of the window is around 50%. 
'''
window_overlap_ratio = 0

'''if the network structure and training strategy is checked as usable, switch 
the leave_out_index_validation parameter to other values from 0 to 7, thus 
conducting a leave-one-out validation for the model. As a result, we don't need 
to split the test data out from the dataset.
''' 
leave_out_index_validation = 0 

'''due to the nature, the EEG signal at time `t` is aligned to the audio signal
at time `t-delay`. 
'''
envelope_delay_time = 0.1 # denoted in seconds


## Network Related Parameters

In [5]:
batch_size                 = 60
epochs                     = 200 # never exceeds 60, indication.
bool_normalization         = True # normalizing the input data or not. 

## Compute on Other Parameters

In [6]:
downsample_frequency = 70
window_length = int(window_time * downsample_frequency)
envelope_delay = int(envelope_delay_time * downsample_frequency)

# Reshape Matlab Data Into Trainable Data

In [9]:
# ===== data generation START ===== #
X_train, Y_train, X_validation, Y_validation = \
    dataGenerator(window_overlap_ratio=window_overlap_ratio,
        window_length=window_length, envelope_delay=envelope_delay, verbose=0,
        tag_EEG=tag_EEG, tag_stimuli=tag_stimuli, tag_attended_track=tag_attended_track,
        EEG_dir=EEG_dir, EEG_test_dir=EEG_test_dir, audio_dir=audio_dir,
        bool_validation=bool_validation, bool_test_set=bool_test_set, bool_envelope=bool_envelope,
        leave_out_index_validation=leave_out_index_validation)
# ===== data generation END ===== #

INFO: Start generating training/validation set with envelope
====> pre_process02_CNN.mat: processing start
====> pre_process03_CNN.mat: processing start
====> pre_process04_CNN.mat: processing start
====> pre_process05_CNN.mat: processing start
====> pre_process06_CNN.mat: processing start
====> pre_process07_CNN.mat: processing start
====> pre_process08_CNN.mat: processing start
====> pre_process09_CNN.mat: processing start
====> pre_process10_CNN.mat: processing start
====> pre_process11_CNN.mat: processing start
====> pre_process12_CNN.mat: processing start
====> pre_process13_CNN.mat: processing start
====> pre_process14_CNN.mat: processing start
====> pre_process15_CNN.mat: processing start
====> pre_process16_CNN.mat: processing start
====> pre_process17_CNN.mat: processing start
INFO: Finished dataset generation.
====> X dims AFTER adding extra dim (for keras)
      X_train      is (3696, 700, 66, 1)
      X_validation is (528, 700, 66, 1)
====> Y dims after applying to_categori

## check whether the outputs are aligned properly



In [ ]:
if verbose:
  sign = 0 # 0 and 1, left or right
  temp=Y_train # Y_validation and Y_train
  print("Printing the output of dataset in order...")
  for i in range(min(1000,temp.shape[0])):
    if int(temp[i-1][sign]) != int(temp[i][sign]):
      print("")
    print(int(temp[i][sign]),end='')

## check whether the inputs are aligned (or seperated) correctly




In [ ]:
if window_overlap_ratio == 0:
  if leave_out_index_validation == 0:
    # print(X_validation.shape)
    X_validation_generated = np.reshape(X_validation[0][0],X_validation[0][0].shape[0])
    # print(X_validation_generated.shape)
    print(X_validation_generated[1:7])
    filename='pre_process02_CNN.mat'
    EEG_all = loadmat(EEG_dir + '/' + filename)
    output_Eeg = EEG_all['output_Eeg']
    # print(output_Eeg[leave_out_index_validation][0].shape)
    print(output_Eeg[leave_out_index_validation][0][envelope_delay][0:6])
    if X_validation_generated[1:7].all() == output_Eeg[leave_out_index_validation][0][7][0:6].all():
      print("The data is aligned when trial " + str(leave_out_index_validation+1) + 
            " is read into the validation set. ")
    del filename, EEG_all, output_Eeg, X_validation_generated

[-0.07177847 -0.10532329 -0.07419017 -0.03526331 -0.01319777 -0.15567087]
[-0.07177847 -0.10532329 -0.07419017 -0.03526331 -0.01319777 -0.15567087]
The data is aligned when trial 1 is read into the validation set. 


# Normalizing Input Data

In [10]:
# normalize input data wrt #channels
# mean, max and min are [66, 700, 1] dimensional vector.
if bool_normalization:
  mega_x = np.concatenate((X_train,X_validation),axis=0)
  X_train[:,:,:,:] = (X_train[:,:,:,:] - np.mean(mega_x[:,:,:,:],axis=(0,3),keepdims=True))/np.std(mega_x[:,:,:,:],axis=(0,3),keepdims=True)
  X_validation[:,:,:,:] = (X_validation [:,:,:,:] - np.mean(mega_x[:,:,:,:],axis=(0,3),keepdims=True))/np.std(mega_x[:,:,:,:],axis=(0,3),keepdims=True)
  #for i in range(X_train.shape[0]):
  #  X_train[i,:,:,:] = (X_train[i,:,:,:] - np.mean(X_train[i,:,:,:]))/(np.max(X_train[i,:,:,:]) - np.min(X_train[i,:,:,:]))
  #for i in range(X_validation.shape[0]):
  #  X_validation[i,:,:,:] = (X_validation[i,:,:,:] - np.mean(X_validation[i,:,:,:]))/(np.max(X_validation[i,:,:,:]) - np.min(X_validation[i,:,:,:]))
  #print('the mean of input (training set) is: ' + str(np.mean(X_train)) + 
  #      '\nthe standard deviation of input (training set) is: ' + str(np.std(X_train)))
  #print('the mean of input (validation set) is: ' + str(np.mean(X_validation)) + 
  #      ',\nthe standard deviation of input (validation set) is: ' + str(np.std(X_validation)))

# Network Related Functions

## Callbacks

### Learning Rate Scheduler Definition



In [11]:
class CustomLearningRateScheduler(keras.callbacks.Callback):

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("Learning rate is %6.4f." % (scheduled_lr))

LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (0, 0.1),
    (10, 0.05),
    (25, 0.025),
    (40, 0.0125),
] 
# Used in pretraining, updated in subject-specific retraining. The learning rate
# is divided by 10 during subject-specific retraining. 

def lr_schedule(epoch, lr):
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
          return LR_SCHEDULE[i][1]
    return lr # if loss stagnate, return lr * 0.1

### Early Stopping Strategy Definition

In [12]:
early_stopping = EarlyStopping(
                                monitor="val_loss",
                                min_delta=0, # no loss reduction
                                patience=10, # 10 non-consecutive epochs
                                verbose=1,
                                mode="min",
                                restore_best_weights=True,
                                )

### Model Checkpoint Definition

In [13]:
checkpoint = ModelCheckpoint(filepath = checkpoint_save_dir + 'pretrained_epoch_{epoch:02d}_valoss{val_loss:.2f}.h5',
                             monitor='val_loss', 
                             verbose=verbose, 
                             save_best_only=True, 
                             mode='min')
# Checkpoints are all saved under the `/checkpoints/` folder. 

### Weight Printing Definition

Print the mean and variance of the weights before and after each epoch. 



In [ ]:
class WeightCallback(tf.keras.callbacks.Callback):
  def on_train_batch_begin(self, batch, logs=None):
    conv_layer = model.get_layer('conv')
    conv_layer_weights = conv_layer.get_weights()
    # print("the shape of weights in conv layer is: (" + \
    #       str(len(conv_layer_weights)) + ',' + \
    #       str(len(conv_layer_weights[0])) + ',' + \
    #       str(len(conv_layer_weights[0][0])) + ',' + \
    #       str(len(conv_layer_weights[0][0][0])) + ',' + \
    #       str(len(conv_layer_weights[0][0][0][0])) + ').')
    conv_layer_weights = np.array(conv_layer_weights,dtype=object)[0]
    conv_layer_weights_flatten=conv_layer_weights.flatten()
    print('===> BEGIN batch: The mean of all weights is: ' + 
          str(np.mean(conv_layer_weights_flatten)) + 
          ', The variances of all weights is: ' + 
          str(np.var(conv_layer_weights_flatten)))
  def on_train_batch_end(self, batch, logs=None):
    conv_layer = model.get_layer('conv')
    conv_layer_weights = conv_layer.get_weights()
    conv_layer_weights = np.array(conv_layer_weights,dtype=object)[0]
    conv_layer_weights_flatten=conv_layer_weights.flatten()
    print('\n===> END   batch: The mean of all weights is: ' + 
          str(np.mean(conv_layer_weights_flatten)) + 
          ', The variances of all weights is: ' + 
          str(np.var(conv_layer_weights_flatten)))
  def on_test_batch_begin(self, batch, logs=None):
    pass
  def on_test_batch_end(self, batch, logs=None):
    pass

weight_print = WeightCallback()

### Callback List Generation

In [14]:
callbacks_list = [CustomLearningRateScheduler(lr_schedule), 
                  checkpoint, 
                  early_stopping] # weight_print

## Network Definition

In [18]:
def baseline_model(window_length, verbose):
  initz = RandomUniform(minval=-0.1, maxval=0.1, seed=None) #intializer
  model = Sequential()
  model.add(Input(shape=[window_length,66,1]))
  model.add(Conv2D(filters=5, name='conv',
                  kernel_size=(9,66),
                  strides=(1,1),
                  padding='valid',
                  activation='relu',
                  ))
  model.add(GlobalAveragePooling2D(name='globalPooling'))
  model.add(Dense(units=5, name='FC1',
                  activation='sigmoid',
                  ))
  model.add(Dense(units=2, name='FC2',
                  activation='softmax',
                  ))
  model.summary()
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                loss='categorical_crossentropy',
                metrics=['accuracy'],
                )
  return model

## Network Check: Can It Overfit on Small Size of Training Samples



This section is for the first sanity check. If the network cannot overfit to small training set (which means it cannot get a ~100% accuracy and a low loss), the network is needed to be redesigned. 

### When Training Samples (Small Size) are Generated Randomly (wrong)

In [ ]:
size_test = batch_size 
X_train_test = np.random.randn(size_test, window_length, 66, 1)
Y_train_test = np.random.randint(2, size=size_test)
Y_train_test = tf.keras.utils.to_categorical(Y_train_test,2)
model_test = baseline_model(window_length, verbose)
history = model_test.fit(X_train_test, Y_train_test,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=2, 
                    shuffle=True)
del size_test, X_train_test, Y_train_test, model_test, history


### When Training Samples (Small Size) are Sampled from the Training Data

In [ ]:
sample_ratio = 0.01 # how much data are sampled from trainings set
_, X_sample, _, Y_sample = train_test_split(X_train, Y_train, 
                                            test_size=sample_ratio)
# print(X_sample[0][0:10][0:10][:])
# print(Y_sample)
model_test = baseline_model(window_length, 1)
history = model_test.fit(X_sample, Y_sample,
                    batch_size=50,
                    epochs=50,
                    verbose=2, 
                    shuffle=True)
del sample_ratio, model_test, X_sample, Y_sample, history

# Pre-Training on Entire Training Data

## Pre-Training: Execution and Save Model in `pretrain.h5`


In [19]:
model = baseline_model(window_length, verbose)
history = model.fit(X_train, Y_train,
                              batch_size=50,
                              epochs=epochs,
                              verbose=1, 
                              callbacks=callbacks_list,
                              validation_data=(X_validation, Y_validation), 
                              shuffle=True)
scores = model.evaluate(X_validation, Y_validation, verbose=verbose)
print("Accuracy: %.2f%%" % (scores[1]*100))
# save model in pretrain.h5
# filepath = model_save_dir + "pretrain.h5"
# model.save(filepath)
# del model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv (Conv2D)                (None, 692, 1, 5)         2975      
_________________________________________________________________
globalPooling (GlobalAverage (None, 5)                 0         
_________________________________________________________________
FC1 (Dense)                  (None, 5)                 30        
_________________________________________________________________
FC2 (Dense)                  (None, 2)                 12        
Total params: 3,017
Trainable params: 3,017
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
Learning rate is 0.1000.
74/74 [==============================] - 13s 172ms/step - loss: 0.7109 - accuracy: 0.5053 - val_loss: 0.7004 - val_accuracy: 0.5000

Epoch 00001: val_loss did not improve from 0.69328
Epoch 2/200
Learning rate is 

In [ ]:
filepath = model_save_dir + "pretrain.h5"
model.save(filepath)

## Redo Pre-Training on Loaded `pretrain.h5` Model (Optional)

If you want to redo the pre-training step, be careful on the learning rate, because once you run `model.fit()`, the learning rate is changed w.r.t. the defined learning rate table. 

So, if you want to **continue** pre-training: 

- change the learning rate to `0.0125` and 
- remove `CustomLearningRateScheduler(lr_schedule)` in `callbacks_list`. 

In [ ]:
filepath = model_save_dir + "pretrain.h5"
pretrain_model=load_model(filepath)

history = pretrain_model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_validation, Y_validation), 
                    callbacks=callbacks_list,
                    shuffle=True)

## Making Plots of Accuracy and/or Loss

In [ ]:
# print(history.history.keys())
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model loss and accuracy')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# save the plot 

## Pre-Training Evaluation

### Evaluation Parameters Generation

In [ ]:
model_filepath_pretrained = experiment_save_dir + 'pretrained.h5'
# evaluate the pretrained model
evaluation_model = load_model(model_filepath_pretrained) 
# evaluation_model.summary()

### Looking into the output of Softmax Layer (two dimensional vector)

In [ ]:
if verbose:
  model.summary()
Y_predict=model.predict(X_validation)
print('===============================')
print('|  prediction output   |  GT  |')
print('===============================')
for i in range(30):
  value = randint(0, X_validation.shape[0])
  print(str(Y_predict[value]) +'\t'+ str(Y_validation[value]))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv (Conv2D)                (None, 692, 1, 5)         2975      
_________________________________________________________________
globalPooling (GlobalAverage (None, 5)                 0         
_________________________________________________________________
FC1 (Dense)                  (None, 5)                 30        
_________________________________________________________________
FC2 (Dense)                  (None, 2)                 12        
Total params: 3,017
Trainable params: 3,017
Non-trainable params: 0
_________________________________________________________________
|  prediction output   |  GT  |
[0.49893415 0.5010659 ]	[0. 1.]
[0.49893415 0.5010659 ]	[0. 1.]
[0.49893415 0.5010659 ]	[0. 1.]
[0.49893415 0.5010659 ]	[0. 1.]
[0.498934 0.501066]	[1. 0.]
[0.49893358 0.5010664 ]	[0. 1.]
[0.4989298 0.5010702]	[1. 0.]


### Looking into the Weights in Convolution Layer




In [ ]:
conv_layer = evaluation_model.get_layer('conv')
conv_layer_weights = conv_layer.get_weights()
print("the shape of weights in conv layer is: (" + \
      str(len(conv_layer_weights)) + ',' + \
      str(len(conv_layer_weights[0])) + ',' + \
      str(len(conv_layer_weights[0][0])) + ',' + \
      str(len(conv_layer_weights[0][0][0])) + ',' + \
      str(len(conv_layer_weights[0][0][0][0])) + ').')
conv_layer_weights = np.array(conv_layer_weights,dtype=object)[0]
# conv_layer_weights = np.array_split(conv_layer_weights, axis=4)
# for i in range(5):
conv_layer_weights_flatten=conv_layer_weights.flatten()
print('The mean of all weights is: ' + 
      str(np.mean(conv_layer_weights_flatten)) + 
      ', The variances of all weights is: ' + 
      str(np.var(conv_layer_weights_flatten)))

the shape of weights in conv layer is: (2,9,66,1,5).
The mean of all weights is: -0.18348497, The variances of all weights is: 45.345387


In [ ]:
a=[[1,2,3,4],[2,2,3,4],[3,2,3,4],[4,2,3,4],[5,2,3,4]]
a[1][:]

4

# Retrain on Subject-Specific Manner

## Preparation for Subject-Specific Retraining

### split dataset into subject specific

In [ ]:
# do subject training and save accuracy
print('BEFORE: shape of training set  : ' + str(X_train.shape) +      ' and ' + str(Y_train.shape))
print('BEFORE: shape of validation set: ' + str(X_validation.shape) + ' and ' + str(Y_validation.shape))
X_train_per_subject = np.reshape(X_train, (number_of_subjects,
                                          int(X_train.shape[0]/number_of_subjects),
                                          X_train.shape[1],
                                          X_train.shape[2],
                                          X_train.shape[3]))
X_validation_per_subject = np.reshape(X_validation, (number_of_subjects, 
                                          int(X_validation.shape[0]/number_of_subjects),
                                          X_validation.shape[1],
                                          X_validation.shape[2],
                                          X_validation.shape[3]))
Y_train_per_subject = np.reshape(Y_train, (number_of_subjects,
                                          int(Y_train.shape[0]/number_of_subjects),
                                          Y_train.shape[1]))
Y_validation_per_subject = np.reshape(Y_validation, (number_of_subjects,
                                          int(Y_validation.shape[0]/number_of_subjects),
                                          Y_validation.shape[1]))
# if verbose:
print('AFTER: shape of training set    : ' + str(X_train_per_subject.shape) + ' and ' + str(Y_train_per_subject.shape))
print('AFTER: shape of validation set  : ' + str(X_validation_per_subject.shape) + ' and ' + str(Y_validation_per_subject.shape))

BEFORE: shape of training set  : (4816, 700, 66, 1) and (4816, 2)
BEFORE: shape of validation set: (688, 700, 66, 1) and (688, 2)
AFTER: shape of training set    : (16, 301, 700, 66, 1) and (16, 301, 2)
AFTER: shape of validation set  : (16, 43, 700, 66, 1) and (16, 43, 2)


## Subject-Specific Retraining Execution

In [ ]:
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (0, 0.01),
    (10, 0.005),
    (25, 0.0025),
    (40, 0.00125),
]
scores=[]
subject_retrained_model=load_model(filepath)
filepath_retrain = model_save_dir + 'model_22_retrain.h5'
for subject_index in range(number_of_subjects):
  print('INFO: Subject retraining. Processing subject: ' + str(subject_index+1))
  X_train_one_subject      = np.reshape(X_train_per_subject[subject_index], 
                                        (X_train_per_subject.shape[1],
                                         X_train_per_subject.shape[2],
                                         X_train_per_subject.shape[3],
                                         X_train_per_subject.shape[4],))
  X_validation_one_subject = np.reshape(X_validation_per_subject[subject_index], 
                                        (X_validation_per_subject.shape[1],
                                         X_validation_per_subject.shape[2],
                                         X_validation_per_subject.shape[3],
                                         X_validation_per_subject.shape[4],))
  Y_train_one_subject      = np.reshape(Y_train_per_subject[subject_index],
                                        (Y_train_per_subject.shape[1],
                                         Y_train_per_subject.shape[2],))
  Y_validation_one_subject = np.reshape(Y_validation_per_subject[subject_index],
                                        (Y_validation_per_subject.shape[1],
                                         Y_validation_per_subject.shape[2],))
  if verbose:
    print("====> shape of training set    : " + str(X_train_one_subject.shape) + ' and ' + str(Y_train_one_subject.shape))
    print("====> shape of validation set  : " + str(X_validation_one_subject.shape) + ' and ' + str(Y_validation_one_subject.shape))
  checkpoint = ModelCheckpoint(filepath_retrain, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
  # callbacks_list = [CustomLearningRateScheduler(lr_schedule), checkpoint]
  callbacks_list = [CustomLearningRateScheduler(lr_schedule),]
  history = subject_retrained_model.fit(X_train_one_subject, Y_train_one_subject,
                    batch_size=int(batch_size/10),
                    epochs=50,
                    verbose=1,
                    validation_data=(X_validation_one_subject, Y_validation_one_subject), 
                    callbacks=callbacks_list,
                    shuffle=True)
  scores.append(model.evaluate(X_validation_one_subject, Y_validation_one_subject, verbose=0))
  subject_retrained_model.save(filepath_retrain)
for i in range(16):
  print('Result: Accuracy of subject ' + str(i+1) + ': ' + str(scores[i][1]))
print('===================================================')
for i in range(16):
  print('Result: Loss     of subject ' + str(i+1) + ': ' + str(scores[i][0]))

In [ ]:
index_need_retrained = [5,6,8,10,11,12,15]
X_train_need_retrained =[]
X_validation_need_retrained=[]
Y_train_need_retrained=[]
Y_validation_need_retrained=[]

def reshape_subject_X(input):
  output = np.array(input)
  output = np.reshape(output, (output.shape[0]*output.shape[1], output.shape[2],output.shape[3], output.shape[4]))
  return output

def reshape_subject_Y(input):
  output = np.array(input)
  output = np.reshape(output, (output.shape[0]*output.shape[1], output.shape[2]))
  return output

for subject_index in range(len(index_need_retrained)):
  print('INFO: Subject retraining. Processing subject: ' + str(subject_index+1))
  X_train_need_retrained.append(X_train_per_subject[index_need_retrained[subject_index]])
  X_validation_need_retrained.append(X_validation_per_subject[index_need_retrained[subject_index]])
  Y_train_need_retrained.append(Y_train_per_subject[index_need_retrained[subject_index]])
  Y_validation_need_retrained.append(Y_validation_per_subject[index_need_retrained[subject_index]])

X_train_need_retrained = np.array(X_train_need_retrained)
X_validation_need_retrained = np.array(X_validation_need_retrained)
Y_train_need_retrained = np.array(Y_train_need_retrained)
Y_validation_need_retrained = np.array(Y_validation_need_retrained)

X_train_need_retrained =reshape_subject_X(X_train_need_retrained)
X_validation_need_retrained=reshape_subject_X(X_validation_need_retrained)
Y_train_need_retrained=reshape_subject_Y(Y_train_need_retrained)
Y_validation_need_retrained=reshape_subject_Y(Y_validation_need_retrained)



In [ ]:
new_model=load_model(filepath_retrain)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [CustomLearningRateScheduler(lr_schedule), checkpoint]
history = new_model.fit(X_train_need_retrained, Y_train_need_retrained,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=2,
                    validation_data=(X_validation_need_retrained, Y_validation_need_retrained), 
                    callbacks=callbacks_list,
                    shuffle=True)


In [ ]:
# model = load_model(filepath_retrain)
new_model.summary()

for i 
scores = new_model.evaluate(X_validation, Y_validation, verbose=verbose)
print("Accuracy: %.2f%%" % (scores[1]*100))

Y_predict=new_model.predict(X_validation_need_retrained)

for i in range(50):
  value = randint(0, X_validation_need_retrained.shape[0])
  print(str(Y_predict[value]) +'\t'+ str(Y_validation[value]))


## Retraining: Evaluation

# Overall Results

## Test Result on Test Set

In [ ]:
if bool_test_set == False:
  pass
